In [1]:
import pandas as pd
import numpy as np
import random
import csv

In [138]:
user_interest=pd.read_csv(r'F:\work\7.21 simrank++\平滑14k=2.csv')
interest_dict=dict()
for i in user_interest.index:
    device=user_interest['deviceid'][i]
    interest_dict.setdefault(device,{})
    for s in range(1,26):
        name='theme_'+str(s)
        j=user_interest[name][i]
        interest_dict[device][str(s)]=j
#interest_dict

In [143]:
#数据转为字典
theme_dict=dict()
essay_theme_dict=dict()
ctr_dict=dict()
with open(r'F:\work\7.21 simrank++\theme_ctr_500_3.csv',encoding='utf_8_sig') as f:
    f.readline()
    for line in f:
        itemid,theme,clicks,exposes,ctr,ctr2=line.strip().split(',')
        essay_theme_dict[itemid]=theme
        theme_dict.setdefault(theme,[])
        theme_dict[theme].append(itemid)
        ctr_dict[itemid]=float(ctr2)
#theme_dict

In [144]:
#avg=np.mean(list(ctr_dict.values()))
avg=0.065
rec_essay_theme=dict()
for i in ctr_dict.keys():
    if ctr_dict[i]>avg:
        e_theme=essay_theme_dict[i]
        rec_essay_theme.setdefault(e_theme,{})
        rec_essay_theme[e_theme][i]=ctr_dict[i]
for i in range(1,26):
    print(i,len(rec_essay_theme[str(i)]))

1 95
2 81
3 87
4 100
5 95
6 97
7 109
8 108
9 84
10 92
11 94
12 85
13 98
14 84
15 86
16 81
17 78
18 74
19 81
20 94
21 88
22 83
23 97
24 87
25 94


In [112]:
tmp=pd.read_csv(r'F:\work\7.21 simrank++\test_expose_1.csv',encoding='utf-8')
test_expose_dict={}
for i in tmp.index:
    device=tmp['deviceid'][i]
    j=tmp['list'][i]
    k=j.strip().split(',')
    test_expose_dict[device]=k
tmp=pd.read_csv(r'F:\work\7.21 simrank++\test_click_1.csv',encoding='utf-8')
test_click_dict={}
for i in tmp.index:
    device=tmp['deviceid'][i]
    j=tmp['list'][i]
    k=j.strip().split(',')
    test_click_dict[device]=k

In [141]:
device_list=[]
for user in interest_dict.keys():
    if user in test_expose_dict.keys():
        if user in test_click_dict.keys():
            device_list.append(user)
sample_users=random.sample(device_list,10000)
len(device_list)

194149

In [145]:
for a in [x/10 for x in range(0,5)]:
    ACC=[]
    RECALL=[]
    expose_ACC=[]
    for user in sample_users:
        score_dict=dict()
        max_label=max(list(interest_dict[user].values()))
        for label in interest_dict[user].keys():
            label_score=interest_dict[user][label]/max_label         #兴趣分
            tmp_dict={v:a*label_score+(1-a)*k for v,k in rec_essay_theme[label].items()}           #兴趣分*ctr
            score_dict.update(tmp_dict)
        rec_sorted=sorted(score_dict.items(),key=lambda x:x[1],reverse=True)
        rec_list=[x[0] for x in rec_sorted]
        rec=sorted(set(rec_list),key=rec_list.index)[:100]
        exposed=[x for x in rec if x in test_expose_dict[user]]
        if len(exposed)>0:
            true=[x for x in rec if x in test_click_dict[user]]
            po=[x for x in test_click_dict[user] if x in rec]
            acc=len(true)/len(rec)
            recall=len(po)/len(test_click_dict[user])
            ACC.append(acc)
            RECALL.append(recall)
            expose_acc=len(true)/len(exposed)
            expose_ACC.append(expose_acc)
#            if sample_users.index(user)%1000==0:
#                print(sample_users.index(user))
    print(a,np.mean(ACC),np.mean(RECALL),np.mean(expose_ACC),2*np.mean(RECALL)*np.mean(expose_ACC)/(np.mean(RECALL)+np.mean(expose_ACC)),len(ACC))

0.0 0.009409845368137324 0.22331956619211174 0.2872460457222154 0.2512807789015317 7049
0.1 0.009262329633272446 0.22033219527744333 0.2842412861677261 0.2482393897929201 7117
0.2 0.008674698795180723 0.20137665790862483 0.26341012027993366 0.22825369468561918 7138


KeyboardInterrupt: 

In [129]:
rec_sorted=sorted(ctr_dict.items(),key=lambda x:x[1],reverse=True)
rec_list=[x[0] for x in rec_sorted]
rec=sorted(set(rec_list),key=rec_list.index)[:100]
ACC=[]
RECALL=[]
expose_ACC=[]
for user in sample_users:
    exposed=[x for x in rec if x in test_expose_dict[user]]
    if len(exposed)>0:
        true=[x for x in rec if x in test_click_dict[user]]
        po=[x for x in test_click_dict[user] if x in rec]
        acc=len(true)/len(rec)
        recall=len(po)/len(test_click_dict[user])
        ACC.append(acc)
        RECALL.append(recall)
        expose_acc=len(true)/len(exposed)
        expose_ACC.append(expose_acc)
    if sample_users.index(user)%5000==0:
        print(sample_users.index(user))
print(np.mean(ACC),np.mean(RECALL),np.mean(expose_ACC),2*np.mean(RECALL)*np.mean(expose_ACC)/(np.mean(RECALL)+np.mean(expose_ACC)),len(ACC))

0
5000
0.005773425880019831 0.11408463313140241 0.2606109505319836 0.15869791893879048 4034


In [23]:
rec_sorted

[('2579724', 1.137539879),
 ('2587519', 1.073158113),
 ('2565573', 1.045520757),
 ('2582571', 0.585139319),
 ('2538707', 0.496789727),
 ('2577093', 0.484771574),
 ('2421680', 0.463031592),
 ('2413149', 0.42023008),
 ('2604889', 0.418147741),
 ('2365814', 0.410574589),
 ('2540673', 0.409139379),
 ('2352829', 0.406999547),
 ('2434064', 0.405729514),
 ('2001602', 0.394612304),
 ('2515540', 0.380861505),
 ('2539418', 0.367542504),
 ('2573715', 0.3630394),
 ('2458558', 0.358098763),
 ('2595081', 0.353419565),
 ('2491412', 0.348870563),
 ('2404501', 0.34530178),
 ('2298230', 0.344293277),
 ('2327663', 0.344099594),
 ('2585596', 0.344001277),
 ('2382875', 0.342953531),
 ('2515915', 0.342531023),
 ('2391433', 0.341769801),
 ('2482245', 0.340663378),
 ('2434830', 0.337975543),
 ('2403236', 0.330826851),
 ('2499493', 0.325988765),
 ('2281592', 0.325630664),
 ('2433490', 0.323440063),
 ('2555687', 0.322033898),
 ('2285262', 0.320360883),
 ('2115294', 0.317978795),
 ('2327653', 0.316847536),
 ('25

In [59]:
rec_sorted

[('2579724', 1.137539879),
 ('2587519', 1.073158113),
 ('2565573', 1.045520757),
 ('2582571', 0.585139319),
 ('2538707', 0.496789727),
 ('2577093', 0.484771574),
 ('2421680', 0.463031592),
 ('2413149', 0.42023008),
 ('2604889', 0.418147741),
 ('2365814', 0.410574589),
 ('2540673', 0.409139379),
 ('2352829', 0.406999547),
 ('2434064', 0.405729514),
 ('2001602', 0.394612304),
 ('2515540', 0.380861505),
 ('2539418', 0.367542504),
 ('2573715', 0.3630394),
 ('2458558', 0.358098763),
 ('2595081', 0.353419565),
 ('2491412', 0.348870563),
 ('2404501', 0.34530178),
 ('2298230', 0.344293277),
 ('2327663', 0.344099594),
 ('2585596', 0.344001277),
 ('2382875', 0.342953531),
 ('2515915', 0.342531023),
 ('2391433', 0.341769801),
 ('2482245', 0.340663378),
 ('2434830', 0.337975543),
 ('2403236', 0.330826851),
 ('2499493', 0.325988765),
 ('2281592', 0.325630664),
 ('2433490', 0.323440063),
 ('2555687', 0.322033898),
 ('2285262', 0.320360883),
 ('2115294', 0.317978795),
 ('2327653', 0.316847536),
 ('25

In [93]:
len(interest_dict)

1703975